In [1]:
# 5. Import the required modules

import networkx as nx
import nx_arangodb as nxadb

from arango import ArangoClient

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

import os
import networkx as nx
import nx_arangodb as nxadb

os.environ["DATABASE_HOST"] = "https://b61c3b83bfe6.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "root"
os.environ["DATABASE_PASSWORD"] = "RHr0KzkRUVlp61IisH8G"
os.environ["DATABASE_NAME"] = "DAC_devops_log"

db = ArangoClient(hosts="https://b61c3b83bfe6.arangodb.cloud:8529") \
    .db(username="root", 
        password="RHr0KzkRUVlp61IisH8G", 
        name="DAC_devops_log",
        verify=True)

[13:45:45 -0500] [INFO]: NetworkX-cuGraph is unavailable: No module named 'cupy'.


In [2]:
arango_graph = ArangoGraph(db)
G_adb = nxadb.DiGraph(name="tasks_sprint1")

[13:45:50 -0500] [INFO]: Graph 'tasks_sprint1' exists.
[13:45:50 -0500] [INFO]: Default node type set to 'task'


In [7]:
arango_graph.schema['Graph Schema']

[{'graph_name': 'emp_interaction1',
  'edge_definitions': [{'edge_collection': 'emp_interaction1_node_to_emp_interaction1_node',
    'from_vertex_collections': ['emp_interaction1_node'],
    'to_vertex_collections': ['emp_interaction1_node']}]},
 {'graph_name': 'employee_interaction',
  'edge_definitions': [{'edge_collection': 'employee_to_employee_interact',
    'from_vertex_collections': ['employee'],
    'to_vertex_collections': ['employee']}]},
 {'graph_name': 'tasks_sprint1',
  'edge_definitions': [{'edge_collection': 'task_depends_on',
    'from_vertex_collections': ['task'],
    'to_vertex_collections': ['task']}]}]

In [8]:
arango_graph.schema

{'Graph Schema': [{'graph_name': 'emp_interaction1',
   'edge_definitions': [{'edge_collection': 'emp_interaction1_node_to_emp_interaction1_node',
     'from_vertex_collections': ['emp_interaction1_node'],
     'to_vertex_collections': ['emp_interaction1_node']}]},
  {'graph_name': 'employee_interaction',
   'edge_definitions': [{'edge_collection': 'employee_to_employee_interact',
     'from_vertex_collections': ['employee'],
     'to_vertex_collections': ['employee']}]},
  {'graph_name': 'tasks_sprint1',
   'edge_definitions': [{'edge_collection': 'task_depends_on',
     'from_vertex_collections': ['task'],
     'to_vertex_collections': ['task']}]}],
 'Collection Schema': [{'collection_name': 'task_depends_on',
   'collection_type': 'edge',
   'edge_properties': [{'name': '_key', 'type': 'str'},
    {'name': '_id', 'type': 'str'},
    {'name': '_from', 'type': 'str'},
    {'name': '_to', 'type': 'str'},
    {'name': '_rev', 'type': 'str'},
    {'name': 'width', 'type': 'int'}],
   'ex

In [ ]:
def create_cpm_table(G_adb):
    """
    Create a table of the critical path method (CPM) and relevant statistics for the given graph.
    Features that will be included for each graph:
    - earliest start time
    - la
    """

    for node in G_adb.nodes():
        story_points = G_adb.nodes[node].get("StoryPoints", "1")  # Default to 1 if missing
        G_adb.nodes[node]["duration"] = int(story_points)  # Ensure integer conversion

    

In [10]:
for node in G_adb.nodes():
    story_points = G_adb.nodes[node].get("StoryPoints", "1")  # Default to 1 if missing
    G_adb.nodes[node]["duration"] = int(story_points)  # Ensure integer conversion

In [11]:
# Find the longest path (Critical Path)
critical_path = nx.algorithms.dag.dag_longest_path(G_adb, weight="duration")

# Compute total project duration
critical_path_length = sum(G_adb.nodes[node]["duration"] for node in critical_path)

print("Critical Path:", critical_path)
print("Total Project Duration:", critical_path_length)

[14:26:46 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.547034740447998s


Critical Path: ['task/TSK-D5031D4A', 'task/TSK-B600DD62', 'task/TSK-29CE08AD', 'task/TSK-89BBB564', 'task/TSK-C2D38D23', 'task/TSK-FBD87AD9', 'task/TSK-7E9F09A9', 'task/TSK-A0500206', 'task/TSK-5A434477']
Total Project Duration: 45


In [12]:
# Step 1: Compute earliest start (ES) and finish (EF)
es = {}  # Earliest Start
ef = {}  # Earliest Finish

for node in nx.topological_sort(G_adb):
    es[node] = max((ef.get(pred, 0) for pred in G_adb.predecessors(node)), default=0)
    ef[node] = es[node] + G_adb.nodes[node]["duration"]

# Step 2: Compute latest finish (LF) and start (LS)
lf = {}  # Latest Finish
ls = {}  # Latest Start

# Start from the last node in the topological order
project_duration = max(ef.values())  # Total project duration
for node in reversed(list(nx.topological_sort(G_adb))):
    lf[node] = min((ls.get(succ, project_duration) for succ in G_adb.successors(node)), default=project_duration)
    ls[node] = lf[node] - G_adb.nodes[node]["duration"]

# Step 3: Compute slack time
slack = {node: ls[node] - es[node] for node in G_adb.nodes()}

# Print Slack times
for node, s in slack.items():
    print(f"Task {node}: Slack Time = {s}")

[14:26:49 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.260451078414917s
[14:27:12 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.654481887817383s


Task task/TSK-991E6E11: Slack Time = 47
Task task/TSK-427652F3: Slack Time = 19
Task task/TSK-5D974201: Slack Time = 13
Task task/TSK-89BBB564: Slack Time = 6
Task task/TSK-CAFF3B09: Slack Time = 36
Task task/TSK-A3E27D20: Slack Time = 50
Task task/TSK-4A3587C9: Slack Time = 34
Task task/TSK-FB3E490E: Slack Time = 18
Task task/TSK-0985DBA9: Slack Time = 29
Task task/TSK-FBED5511: Slack Time = 42
Task task/TSK-10847035: Slack Time = 26
Task task/TSK-E73A7D25: Slack Time = 11
Task task/TSK-93F18D57: Slack Time = 26
Task task/TSK-CB0C0404: Slack Time = 3
Task task/TSK-5A434477: Slack Time = 6
Task task/TSK-42E44473: Slack Time = 14
Task task/TSK-AE8CFDEC: Slack Time = 18
Task task/TSK-DA566BCB: Slack Time = 47
Task task/TSK-85D5D00C: Slack Time = 2
Task task/TSK-AAF06E6A: Slack Time = 3
Task task/TSK-7E9F09A9: Slack Time = 6
Task task/TSK-29CE08AD: Slack Time = 6
Task task/TSK-3D21853C: Slack Time = 29
Task task/TSK-2179FF75: Slack Time = 38
Task task/TSK-0885F3E9: Slack Time = 44
Task ta

In [23]:
slack_score = pd.DataFrame(list(slack.items()), columns=['TaskID', 'slack_time'])
slack_score['TaskID'] = slack_score['TaskID'].str.replace('task/', '')
slack_score = slack_score.sort_values(by='slack_time', ascending=False)
slack_score.head(10)

,TaskID,slack_time
28,TSK-1AA8BEAF,50
5,TSK-A3E27D20,50
56,TSK-91119F5B,47
17,TSK-DA566BCB,47
66,TSK-19A45EE8,47
0,TSK-991E6E11,47
50,TSK-2970AC03,45
24,TSK-0885F3E9,44
40,TSK-BBA87B09,43
9,TSK-FBED5511,42


In [22]:
es_score = pd.DataFrame(list(es.items()), columns=['TaskID', 'earliest_start'])
es_score['TaskID'] = es_score['TaskID'].str.replace('task/', '')
es_score = es_score.sort_values(by='earliest_start', ascending=False)
es_score.head(10)

,TaskID,earliest_start
85,TSK-A8A24727,47
84,TSK-CB0C0404,47
83,TSK-98C9448B,47
82,TSK-85D5D00C,47
79,TSK-5C2187CF,42
78,TSK-C891B895,42
77,TSK-E6CFDD02,42
89,TSK-5A434477,41
73,TSK-983C30DC,34
88,TSK-A0500206,33


In [14]:
def compute_dependency_degrees(graph):
    # Initialize a dictionary to store the dependency degree of each task
    dependency_degrees = {node: None for node in graph.nodes}

    # Recursive function to compute dependency degree with memoization
    def calculate_dependency_degree(task):
        # If the degree is already computed, return it
        if dependency_degrees[task] is not None:
            return dependency_degrees[task]

        # If the task has no dependencies, its dependency degree is 0
        if graph.out_degree(task) == 0:
            dependency_degrees[task] = 0
            return 0

        # Otherwise, calculate the dependency degree for this task
        total_degree = 0
        for dependent_task in graph.predecessors(task):  # Tasks that depend on `task`
            total_degree += 1 + calculate_dependency_degree(dependent_task)

        dependency_degrees[task] = total_degree
        return total_degree

    # Perform topological sorting to ensure we process tasks in dependency order
    for task in nx.topological_sort(graph):
        if dependency_degrees[task] is None:
            calculate_dependency_degree(task)

    # Convert the dependency degrees dictionary to a list of tuples (task, degree)
    dependency_degree_list = [(task, degree) for task, degree in dependency_degrees.items()]

    return dependency_degree_list

In [15]:
dependency_degree = compute_dependency_degrees(G_adb)

dependency_score = pd.DataFrame(dependency_degree, columns=['TaskID', 'Dependency'])
dependency_score['TaskID'] = dependency_score['TaskID'].str.replace('task/', '')
dependency_score = dependency_score.sort_values(by='Dependency', ascending=False)
dependency_score.head(10)

[14:41:49 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.5041379928588867s


,TaskID,Dependency
64,TSK-A0500206,65
20,TSK-7E9F09A9,39
89,TSK-B03EF789,19
55,TSK-FBD87AD9,17
88,TSK-5C2187CF,17
80,TSK-DFB54467,16
53,TSK-983C30DC,13
39,TSK-C2D38D23,13
30,TSK-763CE1C3,11
25,TSK-F7E19B69,7


In [38]:
ls_score = pd.DataFrame(list(ls.items()), columns=['TaskID', 'latest_start'])
ls_score['TaskID'] = ls_score['TaskID'].str.replace('task/', '')
ls_score = ls_score.sort_values(by='latest_start', ascending=False)
ls_score.head(10)

,TaskID,latest_start
61,TSK-1AA8BEAF,50
5,TSK-CB0C0404,50
75,TSK-A3E27D20,50
50,TSK-19A45EE8,50
32,TSK-EBA1C369,49
23,TSK-A6994DD6,49
6,TSK-98C9448B,49
7,TSK-85D5D00C,49
1,TSK-FAAA4D51,49
19,TSK-AE8CFDEC,49


In [39]:
merged_df = dependency_score.merge(es_score, on='TaskID', how='outer')\
                           .merge(slack_score, on='TaskID', how='outer')\
                           .merge(ls_score, on='TaskID', how='outer')

merged_df = merged_df.sort_values(by=['latest_start', 'slack_time'], ascending=[True, True])

merged_df.head(90)

,TaskID,Dependency,earliest_start,slack_time,latest_start
62,TSK-BE4188D4,0,0,0,0
56,TSK-AAF06E6A,0,0,3,3
16,TSK-2B1FAB57,2,5,0,5
42,TSK-89806FDF,0,0,6,6
71,TSK-D5031D4A,0,0,7,7
...,...,...,...,...,...
79,TSK-EBA1C369,0,13,36,49
69,TSK-CB0C0404,0,47,3,50
7,TSK-19A45EE8,0,3,47,50
9,TSK-1AA8BEAF,0,0,50,50


In [ ]:
@tool
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    ######################
    print("1) Generating NetworkX code")

    G_adb = nxadb.DiGraph(name="tasks_sprint1")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
    print('-'*10)
    print(text_to_nx_cleaned)
    print('-'*10)

    ######################

    print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        text_to_nx_final = text_to_nx
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"EXEC ERROR: {e}"

        # TODO: Consider experimenting with a code corrector!
        attempt = 1
        MAX_ATTEMPTS = 3

        # while attempt <= MAX_ATTEMPTS
            # ...

    print('-'*10)
    FINAL_RESULT = local_vars["FINAL_RESULT"]
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################

    print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.
        
        Your response:
    """).content

    return nx_to_text

In [ ]:
xt_to_nx_algorithm_to_text('Perform critical path management on my graph, and tell me the slack time for each task.')

1) Generating NetworkX code
----------
import networkx as nx

# Assuming G_adb is already defined and is a NetworkX DiGraph
# Extract the subgraph related to tasks
task_edges = [(edge['_from'].split('/')[1], edge['_to'].split('/')[1], edge['width']) for edge in G_adb.edges(data=True) if edge[2].get('width') is not None]
task_graph = nx.DiGraph()
task_graph.add_weighted_edges_from(task_edges)

# Perform critical path analysis
length, path = nx.single_source_dijkstra(task_graph, source=None, target=None, weight='weight')

# Calculate slack time for each task
slack_times = {}
for node in task_graph.nodes:
    earliest_start = length[node]
    latest_start = max(length.values()) - nx.single_source_dijkstra_path_length(task_graph, node, weight='weight')
    slack_times[node] = latest_start - earliest_start

# Prepare the final result
FINAL_RESULT = {task: slack for task, slack in slack_times.items()}
----------

2) Executing NetworkX code
EXEC ERROR: tuple indices must be integers or slices

'EXEC ERROR: tuple indices must be integers or slices, not str'